In [1]:
!pip install --quiet elemeta openai


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
from openai import OpenAI
import numpy as np

In [3]:
# defaults to getting the key using os.environ.get("OPENAI_API_KEY")
# if you saved the key under a different environment variable name, you can do something like:
# client = OpenAI(
#   api_key=os.environ.get("CUSTOM_ENV_NAME"),
# )
client = OpenAI()

In [4]:
def ask_gpt(messages,model="gpt-4"):
    ret =  client.chat.completions.create(model=model,
                                          messages=messages
                                          )
    return ret.choices[0].message.content

# The following example is taken from https://platform.openai.com/examples/default-meeting-notes-summarizer

In [5]:

summary_system_prompt="""You will be provided with meeting notes, and your task is to summarize the meeting"""

summary_input_prompt ="""Meeting Date: March 5th, 2050
Meeting Time: 2:00 PM
Location: Conference Room 3B, Intergalactic Headquarters

Attendees:
- Captain Stardust
- Dr. Quasar
- Lady Nebula
- Sir Supernova
- Ms. Comet

Meeting called to order by Captain Stardust at 2:05 PM

1. Introductions and welcome to our newest team member, Ms. Comet

2. Discussion of our recent mission to Planet Zog
- Captain Stardust: "Overall, a success, but communication with the Zogians was difficult. We need to improve our language skills."
- Dr. Quasar: "Agreed. I'll start working on a Zogian-English dictionary right away."
- Lady Nebula: "The Zogian food was out of this world, literally! We should consider having a Zogian food night on the ship."

3. Addressing the space pirate issue in Sector 7
- Sir Supernova: "We need a better strategy for dealing with these pirates. They've already plundered three cargo ships this month."
- Captain Stardust: "I'll speak with Admiral Starbeam about increasing patrols in that area.
- Dr. Quasar: "I've been working on a new cloaking technology that could help our ships avoid detection by the pirates. I'll need a few more weeks to finalize the prototype."

4. Review of the annual Intergalactic Bake-Off
- Lady Nebula: "I'm happy to report that our team placed second in the competition! Our Martian Mud Pie was a big hit!"
- Ms. Comet: "Let's aim for first place next year. I have a secret recipe for Jupiter Jello that I think could be a winner."

5. Planning for the upcoming charity fundraiser
- Captain Stardust: "We need some creative ideas for our booth at the Intergalactic Charity Bazaar."
- Sir Supernova: "How about a 'Dunk the Alien' game? We can have people throw water balloons at a volunteer dressed as an alien."
- Dr. Quasar: "I can set up a 'Name That Star' trivia game with prizes for the winners."
- Lady Nebula: "Great ideas, everyone. Let's start gathering the supplies and preparing the games."

6. Upcoming team-building retreat
- Ms. Comet: "I would like to propose a team-building retreat to the Moon Resort and Spa. It's a great opportunity to bond and relax after our recent missions."
- Captain Stardust: "Sounds like a fantastic idea. I'll check the budget and see if we can make it happen."

7. Next meeting agenda items
- Update on the Zogian-English dictionary (Dr. Quasar)
- Progress report on the cloaking technology (Dr. Quasar)
- Results of increased patrols in Sector 7 (Captain Stardust)
- Final preparations for the Intergalactic Charity Bazaar (All)

Meeting adjourned at 3:15 PM. Next meeting scheduled for March 19th, 2050 at 2:00 PM in Conference Room 3B, Intergalactic Headquarters."""

messages=[
    {"role": "system", "content": summary_system_prompt},

    {"role": "user", "content": summary_input_prompt}
  ]
summary = ask_gpt(messages)
summary

'The meeting on March 5th, 2050 was attended by Captain Stardust, Dr. Quasar, Lady Nebula, Sir Supernova, and new team member Ms. Comet. The mission to Planet Zog was discussed with identified need for improvement in language skills and a suggestion to celebrate with a Zogian food night. The space pirate issue in sector 7 was addressed, with plans to increase patrols and develop new cloaking technology. The team reviewed their second-place performance at the annual Intergalactic Bake-Off and started planning for the upcoming charity fundraiser. Ms. Comet suggested a team-building retreat to the Moon Resort and Spa. Next meeting agenda items include Zogian-English dictionary update, cloaking technology progress, Sector 7 patrol results, and charity bazaar preparations. The next meeting is scheduled for March 19th, 2050.'

In summarization we care mostly about length and coplexity, these give us a notion of how much we managed to trim excess text and how easy it is to understand together they quantify a notion of brevity.

We will create a custom Extractor Runner that extracts both the text's length and complexity.

In [6]:
from elemeta.nlp.extractors.high_level.text_length import TextLength
from elemeta.nlp.extractors.high_level.text_complexity import TextComplexity
from elemeta.nlp.runners.metafeature_extractors_runner import MetafeatureExtractorsRunner
summary_extractors_runner = MetafeatureExtractorsRunner(metafeature_extractors=[TextLength(),TextComplexity()])

In [7]:
summary_extractors_runner.run(summary_input_prompt)

{'text_length': 2635, 'text_complexity': 67.04}

In [8]:
summary_extractors_runner.run(summary)

{'text_length': 829, 'text_complexity': 46.98}

# Lets try to changing the system prompt specifying that we want the summary to be simple and easy to read(to a 5 years old) and notice if the text complexity becomes higher(meaning it is concidered easier to read).

In [9]:
eli5_system_prompt="""You will be provided with meeting notes, and your task is to summarize the meeting
explain like im 5 years old"""
messages=[
    {"role": "system", "content": eli5_system_prompt},
    {"role": "user", "content": summary_input_prompt}
  ]
eli5 = ask_gpt(messages)
eli5

"So, we all met in our cool space office and talked about a lot of things. First, we welcomed a new team member named Ms. Comet. Then we reflected on our trip to a planet called Zog. That trip was fun and successful but we learned that we need to understand their language better, and oh, we really liked their food and we want to have it in our space trip.\n\nLater, Sir Supernova said that the naughty space pirates in a place called Sector 7 have been stealing from us, so we talked about how we can make it harder for them to find us or maybe get more of our space friends to watch the area.\n\nWe also chatted about a big baking contest we took part in and guess what, our team won second place with a Martian Mud Pie. We can't wait for next year because we have a secret plan to win with a special recipe.\n\nAfter that, we looked into what we can do for a charity occasion. We liked the idea of people trying to dunk an alien in a water tank or maybe guessing the names of different stars to w

In [10]:
summary_extractors_runner.run(eli5)

{'text_length': 1417, 'text_complexity': 73.21}

Lets order the output of the first summary and eli5 summary in 1 table(Dataframe) for an easier comparison

In [11]:
pd.DataFrame([
    summary_extractors_runner.run(summary_input_prompt),
    summary_extractors_runner.run(summary),
    summary_extractors_runner.run(eli5)])

,text_length,text_complexity
0,2635,67.04
1,829,46.98
2,1417,73.21


We can see that the eli5 complexity went up meaning it is easier to read, and the text length is reasonably of the same magnitude compared to the original text.